In [1]:
import os
exec(open(os.path.join(os.environ['SPARK_HOME'], 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.cloudera2
      /_/

Using Python version 3.4.3 (default, Nov 17 2016 01:08:31)
SparkSession available as 'spark'.


In [2]:
sc.addPyFile('/opt/graphframes/graphframes-assembly-0.7.0-spark2.3-SNAPSHOT.jar')

from graphframes import * 

In [ ]:
vertices = spark.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("2", "1", "friend"),
  ("1", "3", "friend"),
  ("3", "1", "friend"),
  ("1", "4", "friend"),
  ("4", "1", "friend"),
  ("2", "3", "friend"), 
  ("3", "2", "friend"),
  ("2", "5", "friend"),
  ("5", "2", "friend"),
  ("3", "4", "friend"),
  ("4", "3", "friend"),
  ("3", "5", "friend"),
  ("5", "3", "friend"),
  ("3", "6", "friend"),
  ("6", "3", "friend"),
  ("3", "7", "friend"),
  ("7", "3", "friend")
], ["src", "dst", "relationship"])

gf = GraphFrame(vertices, edges)


In [ ]:
gf.vertices.show()

In [ ]:
gf.edges.show()

In [ ]:
gf.vertices.filter("age > 30").show()

In [ ]:
gf.inDegrees.show()

In [ ]:
gf.inDegrees.filter("inDegree > 2").show()

In [ ]:
results = gf.triangleCount()
results.show()

### Page Rank

In [ ]:
results = gf.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.show()

In [ ]:
results = gf.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.show()

### Connected components

In [ ]:
vertices = spark.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = spark.createDataFrame([
  ("1", "2", "friend"),
  ("1", "4", "friend"),
  ("3", "5", "friend"),
  ("3", "6", "friend"),
  ("3", "7", "friend")
], ["src", "dst", "type"])

reduced_g = GraphFrame(vertices, edges)

In [ ]:
spark.sparkContext.setCheckpointDir("/tmp")
result = reduced_g.connectedComponents()
result.select("id", "component").orderBy("component").show()

###  People you may know

In [6]:
from pyspark.sql.functions import abs, col, explode, split
from pyspark.sql.types import *

In [7]:
dataPath = "./"
graphPathNoOutliers = dataPath + "trainGraph"
usersToPredictPath = dataPath + "prediction.csv"

In [8]:
data = spark.read.format("csv").option("delimiter", "\t")\
    .load(graphPathNoOutliers).withColumnRenamed("_c0", "user").withColumnRenamed("_c1", "friendsString")
    

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, explode

def cutStartEndBrackets(s):
    return s[2:-2]

cutStartEndBracketsUDF = udf(cutStartEndBrackets, StringType())

userFriend = \
    data.select(col("user"), split(cutStartEndBracketsUDF(col("friendsString")), "\),\(").alias("friendsMasks"))\
    .withColumn("friendMask", explode('friendsMasks'))\
    .withColumn("dst", split(col("friendMask"), ",")[0])\
    .withColumn("src", col("user"))\
    .select(col("src").cast("integer"), col("dst").cast("integer"))

userFriendSymmetric = \
    userFriend\
        .withColumn("tmp", col("src"))\
        .withColumn("src", col("dst"))\
        .withColumn("dst", col("tmp"))\
        .select("src", "dst")
        
edges = userFriend.unionAll(userFriendSymmetric)

vertices = edges.select(col("src").alias("id")).distinct()

In [10]:
usersFriends = GraphFrame(vertices, edges)